# Distance matrices

In this notebook, we will  create the distance matrices. We will have to matrices: one for gas cars ane electric cars and the other one for drones. The distance matrices will be used to calculate the cost of the routes.

Each index will be a coordinate in the grid. The distance between two coordinates will be the Havesian distance between them (in the case of drones) and the one given by OSRM (in the case of vehicles).

In [2]:
import pandas as pd

# Data for Gas Cars
gas_cars = {
    "Vehicle": "Gas Car",
    "Freight Rate [COP/km]": 5000,
    "Time Rate [COP/min]": 500,
    "Daily Maintenance [COP/day]": 30000,
    "Recharge/Fuel Cost [COP/(gal or kWh)]": 16000,
    "Recharge/Fuel Time [min/10 percent charge]": 0.1,
    "Avg. Speed [km/h]": None,
    "Gas Efficiency [km/gal]": 10,
}

# Data for Drones
drones = {
    "Vehicle": "Drone",
    "Freight Rate [COP/km]": 500,
    "Time Rate [COP/min]": 500,
    "Daily Maintenance [COP/day]": 3000,
    "Recharge/Fuel Cost [COP/(gal or kWh)]": 220.73,
    "Recharge/Fuel Time [min/10 percent charge]": 2,
    "Avg. Speed [km/h]": 40,
    "Gas Efficiency [km/gal]": None,
    "Electricity Efficency [kWh/km]": 0.15,
}

# Data for Solar EVs
solar_ev = {
    "Vehicle": "Solar EV",
    "Freight Rate [COP/km]": 4000,
    "Time Rate [COP/min]": 500,
    "Daily Maintenance [COP/day]": 21000,
    "Recharge/Fuel Cost [COP/(gal or kWh)]": None,
    "Recharge/Fuel Time [min/10 percent charge]": None,
    "Avg. Speed [km/h]": None,
    "Gas Efficiency [km/gal]": None,
    "Electricity Efficency [kWh/km]": 0.15,
}

# Combine into a dataframe
vehicles_data = pd.DataFrame([gas_cars, drones, solar_ev])

# display the dataframe
vehicles_data

,Vehicle,Freight Rate [COP/km],Time Rate [COP/min],Daily Maintenance [COP/day],Recharge/Fuel Cost [COP/(gal or kWh)],Recharge/Fuel Time [min/10 percent charge],Avg. Speed [km/h],Gas Efficiency [km/gal],Electricity Efficency [kWh/km]
0,Gas Car,5000,500,30000,16000.00,0.1,NaN,10.0,NaN
1,Drone,500,500,3000,220.73,2.0,40.0,NaN,0.15
2,Solar EV,4000,500,21000,NaN,NaN,NaN,NaN,0.15


In [ ]:
C_load_per_min = 500 # [COP/min] Every minute spent at loading/unloading products form the vehicle independently of the vehicle type
loading_speed_5kg_per_min = 1 # [kg/min] Loading speed for 5kg of product

loading_data = {
    "Activity": ["Loading/Unloading"],
    "Cost [COP/min]": [C_load_per_min],
    "Loading Speed [kg/min]": [loading_speed_5kg_per_min]
}


In [65]:
import numpy as np
import requests

clients = pd.read_csv("Clients.csv", delimiter=";|,", engine="python")
client_list = clients.values.tolist()
'''client_dict = {}
i = 0
for key in clients.keys():
    client_dict[key] = i
    i += 1

print(client_dict)'''


depots = pd.read_csv("Depots.csv", delimiter=",|;", engine="python")
depot_list = depots.values.tolist()
'''depot_dict = {}
i = 0
for key in depots.keys():
    depot_dict[key] = i
    i += 1

print(depot_dict)'''
nodes = client_list + depot_list

node_quantity = len(client_list) + len(depot_list)  # Number of nodes in the graph

#cost = np.zeros(((node_quantity,), (node_quantity,)))

cost = [[() for _ in range(node_quantity)] for _ in range(node_quantity)]





def get_distance_duration(origin_map, destination_map):
    # Define the OSRM server URL (use a public instance or your local instance)
    osrm_base_url = "http://router.project-osrm.org/route/v1/driving"

    # Define coordinates: A (longitude, latitude) and B (longitude, latitude)
    coordinate_a = origin_map  # Example: London coordinates
    coordinate_b = destination_map  # Example: nearby location in London

    # Create the request URL
    request_url = f"{osrm_base_url}/{coordinate_a};{coordinate_b}?overview=false"

    # Send a GET request to the OSRM API
    response = requests.get(request_url)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Extract the distance (in meters) and duration (in seconds)
        distance = data['routes'][0]['distance']
        duration = data['routes'][0]['duration']
        distance = distance / 1000
        duration = duration / 60
        return round(distance,2), round(duration,2)
    else:
        return None, None

for i in range(node_quantity-50):
    for j in range(node_quantity):
        if i != j:
            origin = nodes[i]
            destination = nodes[j]
            origin_latitud = origin[-1]
            origin_longitud = origin[-2]
            destination_latitud = destination[-1]
            destination_longitud = destination[-2]
            origin_map = str(origin_longitud) + "," + str(origin_latitud)
            destination_map = str(destination_longitud) + "," + str(destination_latitud)
            cost[i][j] = get_distance_duration(origin_map, destination_map)
            if cost[i][j] == (None, None):
                print("Error in the request")


print(cost)



[[(), (26.95, 33.29), (20.19, 23.53), (18.2, 20.02), (4.2, 6.61), (3.28, 7.34), (6.2, 9.07), (6.04, 9.99), (4.54, 9.57), (15.49, 18.2), (6.67, 9.25), (3.68, 8.19), (9.72, 13.01), (19.48, 23.53), (12.43, 14.93), (18.74, 21.65), (21.23, 25.55), (7.43, 12.23), (8.35, 14.97), (12.43, 17.11), (25.73, 30.76), (15.23, 17.51), (10.23, 13.92), (7.46, 8.11), (0.72, 2.01), (6.99, 10.34), (4.13, 7.5), (5.47, 8.89), (11.86, 16.73), (19.61, 22.35), (9.02, 17.55), (13.64, 19.04), (5.34, 5.83), (3.19, 5.33), (25.84, 32.29), (14.44, 21.35), (20.73, 23.25), (7.98, 9.42), (18.45, 21.11), (10.83, 15.06), (4.35, 9.22), (2.89, 3.72), (10.07, 14.39), (28.42, 35.94), (3.38, 7.57), (9.88, 13.87), (7.89, 8.76), (7.0, 8.25), (21.9, 26.93), (9.02, 17.55), (27.04, 30.59), (18.44, 20.82), (3.47, 6.36), (8.65, 10.75), (7.36, 10.69), (12.02, 16.15), (20.71, 23.51), (7.57, 14.62), (9.75, 13.95), (2.6, 5.82)], [(26.06, 34.84), (), (7.96, 14.29), (14.45, 18.4), (22.95, 30.55), (27.66, 39.14), (27.95, 36.05), (24.38, 32.

In [66]:
for i in range(node_quantity-50, node_quantity-40):
    for j in range(node_quantity):
        if i != j:
            origin = nodes[i]
            destination = nodes[j]
            origin_latitud = origin[-1]
            origin_longitud = origin[-2]
            destination_latitud = destination[-1]
            destination_longitud = destination[-2]
            origin_map = str(origin_longitud) + "," + str(origin_latitud)
            destination_map = str(destination_longitud) + "," + str(destination_latitud)
            cost[i][j] = get_distance_duration(origin_map, destination_map)
            if cost[i][j] == (None, None):
                print("Error in the request")


print(cost)

[[(), (26.95, 33.29), (20.19, 23.53), (18.2, 20.02), (4.2, 6.61), (3.28, 7.34), (6.2, 9.07), (6.04, 9.99), (4.54, 9.57), (15.49, 18.2), (6.67, 9.25), (3.68, 8.19), (9.72, 13.01), (19.48, 23.53), (12.43, 14.93), (18.74, 21.65), (21.23, 25.55), (7.43, 12.23), (8.35, 14.97), (12.43, 17.11), (25.73, 30.76), (15.23, 17.51), (10.23, 13.92), (7.46, 8.11), (0.72, 2.01), (6.99, 10.34), (4.13, 7.5), (5.47, 8.89), (11.86, 16.73), (19.61, 22.35), (9.02, 17.55), (13.64, 19.04), (5.34, 5.83), (3.19, 5.33), (25.84, 32.29), (14.44, 21.35), (20.73, 23.25), (7.98, 9.42), (18.45, 21.11), (10.83, 15.06), (4.35, 9.22), (2.89, 3.72), (10.07, 14.39), (28.42, 35.94), (3.38, 7.57), (9.88, 13.87), (7.89, 8.76), (7.0, 8.25), (21.9, 26.93), (9.02, 17.55), (27.04, 30.59), (18.44, 20.82), (3.47, 6.36), (8.65, 10.75), (7.36, 10.69), (12.02, 16.15), (20.71, 23.51), (7.57, 14.62), (9.75, 13.95), (2.6, 5.82)], [(26.06, 34.84), (), (7.96, 14.29), (14.45, 18.4), (22.95, 30.55), (27.66, 39.14), (27.95, 36.05), (24.38, 32.

In [67]:
for i in range(node_quantity-40, node_quantity-30):
    for j in range(node_quantity):
        if i != j:
            origin = nodes[i]
            destination = nodes[j]
            origin_latitud = origin[-1]
            origin_longitud = origin[-2]
            destination_latitud = destination[-1]
            destination_longitud = destination[-2]
            origin_map = str(origin_longitud) + "," + str(origin_latitud)
            destination_map = str(destination_longitud) + "," + str(destination_latitud)
            cost[i][j] = get_distance_duration(origin_map, destination_map)
            if cost[i][j] == (None, None):
                print("Error in the request")


print(cost)

[[(), (26.95, 33.29), (20.19, 23.53), (18.2, 20.02), (4.2, 6.61), (3.28, 7.34), (6.2, 9.07), (6.04, 9.99), (4.54, 9.57), (15.49, 18.2), (6.67, 9.25), (3.68, 8.19), (9.72, 13.01), (19.48, 23.53), (12.43, 14.93), (18.74, 21.65), (21.23, 25.55), (7.43, 12.23), (8.35, 14.97), (12.43, 17.11), (25.73, 30.76), (15.23, 17.51), (10.23, 13.92), (7.46, 8.11), (0.72, 2.01), (6.99, 10.34), (4.13, 7.5), (5.47, 8.89), (11.86, 16.73), (19.61, 22.35), (9.02, 17.55), (13.64, 19.04), (5.34, 5.83), (3.19, 5.33), (25.84, 32.29), (14.44, 21.35), (20.73, 23.25), (7.98, 9.42), (18.45, 21.11), (10.83, 15.06), (4.35, 9.22), (2.89, 3.72), (10.07, 14.39), (28.42, 35.94), (3.38, 7.57), (9.88, 13.87), (7.89, 8.76), (7.0, 8.25), (21.9, 26.93), (9.02, 17.55), (27.04, 30.59), (18.44, 20.82), (3.47, 6.36), (8.65, 10.75), (7.36, 10.69), (12.02, 16.15), (20.71, 23.51), (7.57, 14.62), (9.75, 13.95), (2.6, 5.82)], [(26.06, 34.84), (), (7.96, 14.29), (14.45, 18.4), (22.95, 30.55), (27.66, 39.14), (27.95, 36.05), (24.38, 32.

In [68]:
for i in range(node_quantity-30, node_quantity-20):
    for j in range(node_quantity):
        if i != j:
            origin = nodes[i]
            destination = nodes[j]
            origin_latitud = origin[-1]
            origin_longitud = origin[-2]
            destination_latitud = destination[-1]
            destination_longitud = destination[-2]
            origin_map = str(origin_longitud) + "," + str(origin_latitud)
            destination_map = str(destination_longitud) + "," + str(destination_latitud)
            cost[i][j] = get_distance_duration(origin_map, destination_map)
            if cost[i][j] == (None, None):
                print("Error in the request")


print(cost)

[[(), (26.95, 33.29), (20.19, 23.53), (18.2, 20.02), (4.2, 6.61), (3.28, 7.34), (6.2, 9.07), (6.04, 9.99), (4.54, 9.57), (15.49, 18.2), (6.67, 9.25), (3.68, 8.19), (9.72, 13.01), (19.48, 23.53), (12.43, 14.93), (18.74, 21.65), (21.23, 25.55), (7.43, 12.23), (8.35, 14.97), (12.43, 17.11), (25.73, 30.76), (15.23, 17.51), (10.23, 13.92), (7.46, 8.11), (0.72, 2.01), (6.99, 10.34), (4.13, 7.5), (5.47, 8.89), (11.86, 16.73), (19.61, 22.35), (9.02, 17.55), (13.64, 19.04), (5.34, 5.83), (3.19, 5.33), (25.84, 32.29), (14.44, 21.35), (20.73, 23.25), (7.98, 9.42), (18.45, 21.11), (10.83, 15.06), (4.35, 9.22), (2.89, 3.72), (10.07, 14.39), (28.42, 35.94), (3.38, 7.57), (9.88, 13.87), (7.89, 8.76), (7.0, 8.25), (21.9, 26.93), (9.02, 17.55), (27.04, 30.59), (18.44, 20.82), (3.47, 6.36), (8.65, 10.75), (7.36, 10.69), (12.02, 16.15), (20.71, 23.51), (7.57, 14.62), (9.75, 13.95), (2.6, 5.82)], [(26.06, 34.84), (), (7.96, 14.29), (14.45, 18.4), (22.95, 30.55), (27.66, 39.14), (27.95, 36.05), (24.38, 32.

In [69]:
for i in range(node_quantity-20, node_quantity-10):
    for j in range(node_quantity):
        if i != j:
            origin = nodes[i]
            destination = nodes[j]
            origin_latitud = origin[-1]
            origin_longitud = origin[-2]
            destination_latitud = destination[-1]
            destination_longitud = destination[-2]
            origin_map = str(origin_longitud) + "," + str(origin_latitud)
            destination_map = str(destination_longitud) + "," + str(destination_latitud)
            cost[i][j] = get_distance_duration(origin_map, destination_map)
            if cost[i][j] == (None, None):
                print("Error in the request")


print(cost)

[[(), (26.95, 33.29), (20.19, 23.53), (18.2, 20.02), (4.2, 6.61), (3.28, 7.34), (6.2, 9.07), (6.04, 9.99), (4.54, 9.57), (15.49, 18.2), (6.67, 9.25), (3.68, 8.19), (9.72, 13.01), (19.48, 23.53), (12.43, 14.93), (18.74, 21.65), (21.23, 25.55), (7.43, 12.23), (8.35, 14.97), (12.43, 17.11), (25.73, 30.76), (15.23, 17.51), (10.23, 13.92), (7.46, 8.11), (0.72, 2.01), (6.99, 10.34), (4.13, 7.5), (5.47, 8.89), (11.86, 16.73), (19.61, 22.35), (9.02, 17.55), (13.64, 19.04), (5.34, 5.83), (3.19, 5.33), (25.84, 32.29), (14.44, 21.35), (20.73, 23.25), (7.98, 9.42), (18.45, 21.11), (10.83, 15.06), (4.35, 9.22), (2.89, 3.72), (10.07, 14.39), (28.42, 35.94), (3.38, 7.57), (9.88, 13.87), (7.89, 8.76), (7.0, 8.25), (21.9, 26.93), (9.02, 17.55), (27.04, 30.59), (18.44, 20.82), (3.47, 6.36), (8.65, 10.75), (7.36, 10.69), (12.02, 16.15), (20.71, 23.51), (7.57, 14.62), (9.75, 13.95), (2.6, 5.82)], [(26.06, 34.84), (), (7.96, 14.29), (14.45, 18.4), (22.95, 30.55), (27.66, 39.14), (27.95, 36.05), (24.38, 32.

In [70]:
for i in range(node_quantity-10, node_quantity):
    for j in range(node_quantity):
        if i != j:
            origin = nodes[i]
            destination = nodes[j]
            origin_latitud = origin[-1]
            origin_longitud = origin[-2]
            destination_latitud = destination[-1]
            destination_longitud = destination[-2]
            origin_map = str(origin_longitud) + "," + str(origin_latitud)
            destination_map = str(destination_longitud) + "," + str(destination_latitud)
            cost[i][j] = get_distance_duration(origin_map, destination_map)
            if cost[i][j] == (None, None):
                print("Error in the request")


print(cost)

[[(), (26.95, 33.29), (20.19, 23.53), (18.2, 20.02), (4.2, 6.61), (3.28, 7.34), (6.2, 9.07), (6.04, 9.99), (4.54, 9.57), (15.49, 18.2), (6.67, 9.25), (3.68, 8.19), (9.72, 13.01), (19.48, 23.53), (12.43, 14.93), (18.74, 21.65), (21.23, 25.55), (7.43, 12.23), (8.35, 14.97), (12.43, 17.11), (25.73, 30.76), (15.23, 17.51), (10.23, 13.92), (7.46, 8.11), (0.72, 2.01), (6.99, 10.34), (4.13, 7.5), (5.47, 8.89), (11.86, 16.73), (19.61, 22.35), (9.02, 17.55), (13.64, 19.04), (5.34, 5.83), (3.19, 5.33), (25.84, 32.29), (14.44, 21.35), (20.73, 23.25), (7.98, 9.42), (18.45, 21.11), (10.83, 15.06), (4.35, 9.22), (2.89, 3.72), (10.07, 14.39), (28.42, 35.94), (3.38, 7.57), (9.88, 13.87), (7.89, 8.76), (7.0, 8.25), (21.9, 26.93), (9.02, 17.55), (27.04, 30.59), (18.44, 20.82), (3.47, 6.36), (8.65, 10.75), (7.36, 10.69), (12.02, 16.15), (20.71, 23.51), (7.57, 14.62), (9.75, 13.95), (2.6, 5.82)], [(26.06, 34.84), (), (7.96, 14.29), (14.45, 18.4), (22.95, 30.55), (27.66, 39.14), (27.95, 36.05), (24.38, 32.

In [74]:
prueba = cost.copy()
for i in range(node_quantity):
    for j in range(node_quantity):
        if i == j:
            prueba[i][j] = (-1, -1)
print(prueba)

[[(-1, -1), (26.95, 33.29), (20.19, 23.53), (18.2, 20.02), (4.2, 6.61), (3.28, 7.34), (6.2, 9.07), (6.04, 9.99), (4.54, 9.57), (15.49, 18.2), (6.67, 9.25), (3.68, 8.19), (9.72, 13.01), (19.48, 23.53), (12.43, 14.93), (18.74, 21.65), (21.23, 25.55), (7.43, 12.23), (8.35, 14.97), (12.43, 17.11), (25.73, 30.76), (15.23, 17.51), (10.23, 13.92), (7.46, 8.11), (0.72, 2.01), (6.99, 10.34), (4.13, 7.5), (5.47, 8.89), (11.86, 16.73), (19.61, 22.35), (9.02, 17.55), (13.64, 19.04), (5.34, 5.83), (3.19, 5.33), (25.84, 32.29), (14.44, 21.35), (20.73, 23.25), (7.98, 9.42), (18.45, 21.11), (10.83, 15.06), (4.35, 9.22), (2.89, 3.72), (10.07, 14.39), (28.42, 35.94), (3.38, 7.57), (9.88, 13.87), (7.89, 8.76), (7.0, 8.25), (21.9, 26.93), (9.02, 17.55), (27.04, 30.59), (18.44, 20.82), (3.47, 6.36), (8.65, 10.75), (7.36, 10.69), (12.02, 16.15), (20.71, 23.51), (7.57, 14.62), (9.75, 13.95), (2.6, 5.82)], [(26.06, 34.84), (-1, -1), (7.96, 14.29), (14.45, 18.4), (22.95, 30.55), (27.66, 39.14), (27.95, 36.05),

In [75]:
import csv

# Example matrix where each element is a tuple of two numbers
matrix = prueba

# Write the matrix to a CSV file
with open("matrix.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    for row in matrix:
        # Convert each tuple to a string and write as a row
        writer.writerow([str(cell) for cell in row])

print("CSV file 'matrix.csv' created successfully.")


CSV file 'matrix.csv' created successfully.


In [76]:
# Read the matrix from the CSV file and convert it back to a list of lists of tuples
reconstructed_matrix = []
with open("matrix.csv", "r") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        # Convert each string representation of a tuple back to a tuple
        reconstructed_row = [eval(cell) for cell in row]
        reconstructed_matrix.append(reconstructed_row)

print("Reconstructed Matrix:")
print(reconstructed_matrix)


Reconstructed Matrix:
[[(-1, -1), (26.95, 33.29), (20.19, 23.53), (18.2, 20.02), (4.2, 6.61), (3.28, 7.34), (6.2, 9.07), (6.04, 9.99), (4.54, 9.57), (15.49, 18.2), (6.67, 9.25), (3.68, 8.19), (9.72, 13.01), (19.48, 23.53), (12.43, 14.93), (18.74, 21.65), (21.23, 25.55), (7.43, 12.23), (8.35, 14.97), (12.43, 17.11), (25.73, 30.76), (15.23, 17.51), (10.23, 13.92), (7.46, 8.11), (0.72, 2.01), (6.99, 10.34), (4.13, 7.5), (5.47, 8.89), (11.86, 16.73), (19.61, 22.35), (9.02, 17.55), (13.64, 19.04), (5.34, 5.83), (3.19, 5.33), (25.84, 32.29), (14.44, 21.35), (20.73, 23.25), (7.98, 9.42), (18.45, 21.11), (10.83, 15.06), (4.35, 9.22), (2.89, 3.72), (10.07, 14.39), (28.42, 35.94), (3.38, 7.57), (9.88, 13.87), (7.89, 8.76), (7.0, 8.25), (21.9, 26.93), (9.02, 17.55), (27.04, 30.59), (18.44, 20.82), (3.47, 6.36), (8.65, 10.75), (7.36, 10.69), (12.02, 16.15), (20.71, 23.51), (7.57, 14.62), (9.75, 13.95), (2.6, 5.82)], [(26.06, 34.84), (-1, -1), (7.96, 14.29), (14.45, 18.4), (22.95, 30.55), (27.66, 3

In [79]:
import requests

print(reconstructed_matrix[50][51])

origin = nodes[50]
destination = nodes[51]
origin_latitud = origin[-1]
origin_longitud = origin[-2]
destination_latitud = destination[-1]
destination_longitud = destination[-2]
origin_map = str(origin_longitud) + "," + str(origin_latitud)
destination_map = str(destination_longitud) + "," + str(destination_latitud)


def get_distance_duration(origin_map, destination_map):
    # Define the OSRM server URL (use a public instance or your local instance)
    osrm_base_url = "http://router.project-osrm.org/route/v1/driving"

    # Define coordinates: A (longitude, latitude) and B (longitude, latitude)
    coordinate_a = origin_map  # Example: London coordinates
    coordinate_b = destination_map  # Example: nearby location in London

    # Create the request URL
    request_url = f"{osrm_base_url}/{coordinate_a};{coordinate_b}?overview=false"

    # Send a GET request to the OSRM API
    response = requests.get(request_url)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Extract the distance (in meters) and duration (in seconds)
        distance = data['routes'][0]['distance']
        duration = data['routes'][0]['duration']
        distance = distance / 1000
        duration = duration / 60
        return distance, duration
    else:
        return None, None


distance, duration = get_distance_duration(origin_map, destination_map)
print(distance)
print(duration)


(15.35, 22.15)
15.3475
22.154999999999998
